# Importing the Dataset and Packages

The powershell in the assosiated GitHub repository allows for this notebook to download the datasetfor this project, provided you complete the ".env" file with your kaggle API key, and file path.

This script was written, so that this notebook may be run, tested, and modified either in the kaggle environment, or on a configured windows machine.

In [ ]:
import re
import warnings
import math
import dotenv
import kagglehub
import os
import subprocess
import ipywidgets


import sklearn as sk
import pandas as pd
import tensorflow as tf
import keras as ks
import matplotlib as mlt
import matplotlib.pyplot as plt
import seaborn as sb


from scipy.stats import anderson, lognorm, probplot, shapiro
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


try:
    dotenv.load_dotenv()
except:
    print("--Dotenv not loaded--")


# Check for Kaggle environment and set the file path
if os.path.exists("/kaggle/input/churn-modelling/Churn_Modelling.csv"):
    # Kaggle
    file_path = "/kaggle/input/churn-modelling/Churn_Modelling.csv"
else:
    # Local
    file_path = (str((os.getenv("LOCAL_FILE_LOCATION"))))

# Load Dataset
try:
    df = pd.read_csv(file_path)
    print("Dataset Loaded Successfully!")
except FileNotFoundError:
    print(f"Error: File not found at : file_path")
    try:
        print("Attempting to run download_data_ps1")
        path = os.getenv("SCRIPT_PATH")
        subprocess.run(["powershell", "-ExecutionPolicy", "Bypass", "-File", path],
                       check = True, capture_output =  True, text = True)
        print("Powershell Download Script Run Successfully. Now attempting to reload dataset...")
        df = pd.read_csv(file_path)
        if df is not None and not df.empty:
            print("Dataset Loaded Successfully")
        else:
            print("Data not loaded")
    except Exception as e:
        print(f"Error running powershell script: {e}")
        df = None

# Display the first few rows of the dataset
if df is not None:
    display(df.head())

# Cleaning Dataset

The target variable is the last variable in this dataset. It is called "Exited". It is binary, where a 1 represents a customer closing thier account, and a 0 represents a retained customer.

Let's preview the data in order to understand what we have to work with.

First, I will drop the insignificant variables, which are the "RowNumber", "CustomerId", and "Surname" variables. They are arbitrary, and not useful for our algorithm.

In [ ]:
RANDOM = 379

X = df.iloc[:, 3:-1]
Y = df.iloc[:,-1:]

display(X.head())
display(Y.head())

display(f"{X.shape=}")
display(f"{Y.shape=}")

As you can see, we have 10,000 observations for both the predictor and target variables.

Now, we will check the dataset for any Null values and duplicates.

In [ ]:
new_df = X.copy()
new_df['Exited'] = Y

print(df.isna().sum(), '\n')
print(f"Duplicate Count   ", df.duplicated().sum())

In [ ]:
pattern = r'\?'
null_df = df["Surname"].astype(str).str.contains(pattern)
mark_count = 0
for val in null_df: 
    if val : mark_count += 1
display(f"The number of question marks appearing in the surname column is : {mark_count}.")

In the "Surname" column, there are rare instances of "?" appearing. This indicates that there is missing or incomplete names. This is not concerning, because the "Surname" variable will be discarded when we build our model. 

Since there are no concerning NA values or duplicates, we can proceed with encoding.

# Exploratory Data Analysis

### Basic Plots

In this section, I will try to get a basic idea of what this dataset looks like.

# Model Selection

### Distribution

First, we will examine the distributions of several variables in the dataset.

In [ ]:
df.hist(bins = 30, figsize=(22, 10), color = 'green');

Since this dataset is so large, it can be hard to tell exactly whether or not a distribution is normal. In order to proceed with my analysis, I will check for normality using the Anderson test.

I picked the Anderson test for the creditscore variable since more weight is given to the tails of the distribution. This is Useful in this situation because of the sharp uptick at the right-tail of 'CreditScore'. 

Additionally, the Anderson test is suitable for large sets of observations. The Shapiro-Wilk test for normality would probably determine the 'CreditScore' variable to be normally distributed, since it is an unsuitable test for large inputs.


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    # Suppress warning. 
    # The warning states the shapiro wilk test is inaccurate for N > 5000.
    # Current N is 10000
    stat, p = shapiro(X['CreditScore'])
    print(f"Shapiro-Wilk Test: Stat = {round(stat, 3)}, p-val = {p}\n")


result = anderson(X['CreditScore'])
print(f"Anderson Test: test-stat = {round(result.statistic, 3)}, Critical Values = {result.critical_values}")

The Anderson test statistic is 5.458. The critical values [0.576 0.656 0.787 0.918 1.092] correspond to significance levels [15%, 10%, 5%, 2.5% 1%]. Since the test statistic is greater than all critical values, we reject the null hypothesis. 

Compare this to the Shapiro-Wilk test. Since its p-value is less than 0.05, we would reject the null hypothesis and determine that the data is normally distributed. However, since the Shapiro test is not suitable for this dataset, we will disregard it.

Therefore, the Anderson test determined that 'CreditScore' is not normally distributed.

In [ ]:
probplot(X['CreditScore'], dist = "norm", plot = plt)
plt.title("Q-Q Plot of CreditScore")
plt.show()

In [ ]:
probplot(X['EstimatedSalary'], dist = "norm", plot = plt)
plt.title("Q-Q Plot of EstimatedSalary")
plt.show()

When looking at the "CreditScore" Q-Q Plot, it is obviously not normally distributed. Also, I threw in the "EstimatedSalary" Q-Q plot. It follows a more uniform like distribution.

We can visually inspect the previously produced histograms to determine that the "Balance" variable will behave simalarly.


### Why use a neural network

Since the data is not normally distributed, we need a non-parametric model. Since the output is binary, we could use a multiple logistic regression model. Logistic regression does not require a normal distribution. However, it would require manual feature engineering to capture non-linear relationships, whereas a neural network is capable of automatically learning these patterns. This greatly cuts down on the amount of pre-processing work, since we only need to encode the categorical variables and scale all the features.

The non-linear nature of this data, non-normal distribution, as well as the mix of categorical and quantitative data suggests that a neural network is a good model to build.

# Building The Neural Network

## Label Encoding
Since we have two categorical variables, we must encode them before proceeding with the analysis.

In [ ]:
adjust_length = 17 # offset for output formatting

for col in X:
    print(f"{col.ljust(adjust_length)} : {X[col].dtypes}")
print(f"{("Exited").ljust(adjust_length)} : {Y['Exited'].dtypes}")

The "Gender" variable will be encoded. A 0 represents the "Female" gender, a 1 represents the "Male" Gender.

Similarly, the "Geography" variable will be encoded. Each geographical location will recieve its own binary column, with a 1 occurring in the column where the observation is located.

In [ ]:
Encoder = LabelEncoder()
X['Gender'] = Encoder.fit_transform(X.iloc[:, 2]).astype(int)

display((X.loc[:, 'CreditScore':'Gender']).iloc[0 : 5])


print(f"{str(X['Gender'].name)} : {X['Gender'].dtypes}")


Now, the "Gender" variable is represented as an integer.

Next, we must encode the "Geography" variable.

In [ ]:
X = pd.get_dummies(X, columns=['Geography'], drop_first=True)

# Cast geography variables to integers
for col in X.columns:
    if 'Geography_' in col:
        X[col] = X[col].astype('int64')

for col in X:
    print(f"{col.ljust(adjust_length)} : {X[col].dtypes}")

Now all of the variables are represented numerically.

In [ ]:
fin_df = X.copy()
fin_df['Exited'] = Y

display(fin_df.iloc[0 : 5])

## Feature Scaling


## Correlation Analysis

## Building the Neural Network

## Optimizing the Model

# Analyze Model

# Conclusions


to-do: 

       - Perform basic EDA BEFORE encoding / scaling
       - Scale features
       - Perform EDA AFTER encoding/scaling
       - Build neural network
       - Optimize NN
       - Analyze NN accuracy
       - Review + Document
       - Publish

